In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib
from numpy import linalg as LA
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
from PIL import Image
import os

In [2]:
class gaussian_sample_layer(nn.Module):

    def __init__(self, latent_dim):
        super(gaussian_sample_layer, self).__init__()
        self.latent_dim = latent_dim
        self.L = 1
        
    def forward(self, mu, sigma):
        epsilon_dist = torch.distributions.MultivariateNormal(torch.zeros(self.latent_dim),torch.eye(self.latent_dim))
        epsilon = epsilon_dist.sample((self.L,))
        epsilon = torch.sum(epsilon, dim=0) / self.L
        a = mu + epsilon * sigma
        return a

In [3]:
class variational_autoencoder(torch.nn.Module):

    def __init__(self):
        super(variational_autoencoder, self).__init__()
        input_size = 784
        output_size = 784
        self.latent_dim = 2
        self.mlp1 = nn.Linear(input_size, 128)
        self.mu = nn.Linear(128, self.latent_dim)
        self.sigma = nn.Linear(128, self.latent_dim)
        self.gaussian = gaussian_sample_layer(self.latent_dim)
        self.mlp4 = nn.Linear(self.latent_dim, 128)
        self.out = nn.Linear(128, output_size)
        self.batch_size = 8

    def forward(self, x):
        h1 = F.sigmoid(self.mlp1(x))
        mu = F.sigmoid(self.mu(h1))
        sigma = F.sigmoid(self.sigma(h1))
        z = self.gaussian(mu, sigma)
        h4 = F.tanh(self.mlp4(z))
        y_hat = F.relu(self.out(h4))
        return y_hat, mu, sigma
    
    def loss(self, x, y, beta=0.0001):
        y_hat, mu, sigma = self.forward(x)
        c = nn.MSELoss()
        l = c(y_hat, y) - beta * 1/2 * torch.sum(1 + torch.log(sigma**2) - mu**2 - sigma**2)
        return l
    
    def decoder(self, x):
        f = F.relu(self.out(F.tanh(self.mlp4(x))))
        return f

In [4]:
vae = variational_autoencoder()
optimizer = optim.Adam(vae.parameters(), lr=3e-4)

In [5]:
batch_size = 8
train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
trainset = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

In [6]:
for epoch in range(100):
    cnt = 0
    l = 0
    for data in trainset:
        data = data[0].squeeze()
        x = torch.reshape(data, (batch_size, 784))
        y = x.clone()
        optimizer.zero_grad()
        y_hat, _, _ = vae(x)
        loss = vae.loss(x, y)
        loss.backward()
        optimizer.step()
        l += loss
        cnt += 1
    print(l)
    '''if cnt % 600 == 0:
            cnt = 0
            print(l / 600)
            l = 0'''

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor(544.1953, grad_fn=<AddBackward0>)
tensor(523.0386, grad_fn=<AddBackward0>)
tensor(513.8260, grad_fn=<AddBackward0>)
tensor(502.2106, grad_fn=<AddBackward0>)
tensor(490.5596, grad_fn=<AddBackward0>)
tensor(484.1887, grad_fn=<AddBackward0>)
tensor(479.0475, grad_fn=<AddBackward0>)
tensor(474.9055, grad_fn=<AddBackward0>)
tensor(470.0655, grad_fn=<AddBackward0>)
tensor(465.0965, grad_fn=<AddBackward0>)
tensor(459.9419, grad_fn=<AddBackward0>)
tensor(456.2365, grad_fn=<AddBackward0>)
tensor(450.8407, grad_fn=<AddBackward0>)
tensor(448.2797, grad_fn=<AddBackward0>)
tensor(446.8343, grad_fn=<AddBackward0>)
tensor(442.5983, grad_fn=<AddBackward0>)
tensor(438.1119, grad_fn=<AddBackward0>)
tensor(436.9942, grad_fn=<AddBackward0>)
tensor(435.7184, grad_fn=<AddBackward0>)
tensor(434.1134, grad_fn=<AddBackward0>)
tensor(432.0978, grad_fn=<AddBackward0>)
tensor(431.2682, grad_fn=<AddBackward0>)
tensor(430.0606, grad_fn=<AddBackward0>)
tensor(428.2201, grad_fn=<AddBackward0>)
tensor(425.8666,

In [7]:
test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
testset = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

In [8]:
results = []
cnt = 0
os.makedirs("img", exist_ok=True)
for test in testset:
    data = test[0].squeeze()
    x = torch.reshape(data, (batch_size, 784))
    out = vae(x)[0].detach().numpy()
    cnt += 1
    for i in range(data.shape[0]):
        plt.imsave('./img/' + str(cnt) + str(i) + 'org.png', data[i], cmap='gray')
        plt.imsave('./img/' + str(cnt) + str(i) + 'recovered.png', out[i].reshape(28, 28), cmap='gray')

In [9]:
weights = []
for w in vae.parameters():
    weights.append(w)
weights = np.array(weights)

In [11]:
os.makedirs("generated", exist_ok=True)
for i in range(1000):
    dist = torch.distributions.MultivariateNormal(torch.zeros(2),torch.eye(2))
    z = dist.sample()
    plt.imsave('./generated/' + str(i)+'.png', vae.decoder(z).detach().numpy().reshape(28, 28), cmap='gray')

In [35]:
steps = 10
x = np.linspace(0, 1, num=steps)
y = np.linspace(0, 1, num=steps)
z = []
for xval in x:
    for yval in y:
        z.append((xval, yval))
z = np.array(z)

In [36]:
os.makedirs("grid", exist_ok=True)
cnt = 0
for pair in z:
    cnt += 1
    plt.imsave('./grid/' + str(cnt)+'.png', vae.decoder(torch.from_numpy(pair).float()).detach().numpy().reshape(28, 28), cmap='gray')